# Cython Optimize Zeros API
The Cython Optimize Zeros API let's users interface with the root finders for scalar functions in the SciPy Optimize package. In order to use it, users must write and compile Cython code. There are several ways to do that, which are described in the Cython Documentation.

## Cython Jupyter Extension
The Cython Jupyter extension is one method of compiling Cython code that is described in the Cython Documentation. To enable it first use the following magic:

    %load_ext Cython

In [1]:
%load_ext Cython

## Compiling Cython in Jupyter
To compile Cython in Jupyter start with `%%cython` magic in the cell you want to compile. The following test is copied verbatim from the Cython Documentation

In [2]:
%%cython

cdef int a = 0
for i in range(10):
    a += i
print(a)

45


## SciPy Cython Special API
The Scipy Optimize Zeros API was inspired by the Cython Special API. The following test is from the SciPy Specials documentation.

In [3]:
%%cython

cimport scipy.special.cython_special as csc

cdef:
    double x = 1
    double complex z = 1 + 1j
    double si, ci, rgam
    double complex cgam

rgam = csc.gamma(x)
print(rgam)
cgam = csc.gamma(z)
print(cgam)
csc.sici(x, &si, &ci)
print(si, ci)


1.0
(0.49801566811835557-0.15494982830181037j)
0.9460830703671831 0.33740392290096816


## Example Usage of Cython Optimize Zeros API
The following example using the `double (*callback_type_tuple)(double x, tuple args)` callback signature with the `brentq` root-finder to find the root of the function $f(x) = 1 - \exp(-(x - 0.7))$.

1. first import the `cython_optimize` package using `cimport`
2. then write the callback in Cython with the matching signature
3. finally call the root-finder with it's expected signature

In [4]:
%%cython

import math
cimport scipy.optimize.cython_optimize as coz

cdef double f(double x, tuple args):
    return 1.0 - math.exp(-(x - 0.7))


print(coz.zeros.brentq(f, 0.5, 1, (), 1e-3, 1e-3, 10))

0.6999942848231314


Validate the result with the existing method.

In [5]:
from scipy.optimize import brentq

brentq(lambda x: 1.0 - math.exp(-(x - 0.7)), 0.5, 1.0, (), 1e-3, 1e-3, 10)

0.6999942848231314

Try the signature `double (*callback_type)(double x, void *args)`. This callback takes a double and a `struct` we call `test_params` that in this example has one extra parameter, `constant`, that is used in the function. The `brentq` root-finder corresponding to this callback is in `scipy.optimize.cython_optimize.zeros_struct`.

In [6]:
%%cython --annotate

import math
cimport scipy.optimize.cython_optimize as coz

ctypedef struct test_params:
    double C0
    double C1
    

cdef double f(double x, void *args):
    cdef test_params *myargs = <test_params *> args
    return myargs.C0 - math.exp(-(x - myargs.C1))


cdef test_params myargs
myargs.C0 = 1.0
myargs.C1 = 0.7

print(coz.zeros_struct.brentq(f, 0.5, 1, <test_params *> &myargs, 1e-3, 1e-3, 10))

0.6999942848231314


The last signature is `double (*callback_type_array)(int n, double *args)` takes an array and its length as inputs. The solvers for this signature are in `scipy.optimize.cython_optimize.zeros_array`. When writing the callback for this signature, the length of the array, `n`, is probably not used directly, but it must still be passed to the solver along with the array of extra arguments. The independent variable should be prepended to the beginning of the array when it is used in the callback, but **not** when it is passed to the solver.

In [7]:
%%cython --annotate

import math
cimport cpython
cimport scipy.optimize.cython_optimize as coz

ctypedef struct test_params:
    double constant

DEF N = 2  # define a compile time constant using "DEF"

cdef double f(int n, double* args):
    # the first "arg" must be the independent variable "x"
    return args[1] - math.exp(-(args[0] - args[2]))


cdef int n = N  # the number of "extra arguments"
# use the compile time constant "N" to allocate the array
cdef double[N] myargs = (1.0, 0.7)  # extra arguments

print(coz.zeros_array.brentq(f, 0.5, 1, n, myargs, 1e-3, 1e-3, 10))

0.6999942848231314
